In [1]:
#!/usr/bin/env pyton
#import sys
#import subprocess as sp
import re

######### works, but some genes are not included
sam_in = input('Sample names (with _all) seperated with a comma:\n').strip()
sam = re.split(r'\s*\,+\s*', sam_in)
stan = input('Normalization standard:\n')
utr = input('5utr, 3utr, or CDS\n')

def main():
    for ss in sam:
        cdna_sum(ss)


def norm(sam, stan):
    # get the normalization ratio
    geex, struc, cose, coan, coto, mir, u21 = 0, 0, 0, 0, 0, 0, 0
    file = sam+'/sum_'+sam+'.txt'
    with open (file) as f:
        for line in f:
            #line = re.sub(r'\s+$', r'', line)
            line = line.rstrip()
            if re.search(r'^geex\t\S+\t\S+\t\S+', line):
                mch = re.search(r'^geex\t\S+\t\S+\t(\S+)', line) # all
                geex = float(mch.group(1))
            if (re.search(r'^struc\t\S+\t\S+\t\S+', line)):
                mch = re.search(r'^struc\t(\S+)\t\S+\t\S+', line) # sense
                struc = float(mch.group(1))
            if (re.search(r'^coding\t\S+\t\S+\t\S+', line)):
                mch = re.search(r'^coding\t(\S+)\t(\S+)\t(\S+)', line)
                cose = float(mch.group(1)); coan = float(mch.group(2)); coto = float(mch.group(3))
            if (re.search(r'^miRNA\t\S+\t\S+\t\S+', line)):
                mch = re.search(r'miRNA\t(\S+)\t\S+\t\S+', line) # sense
                mir = float(mch.group(1))
            if (re.search(r'^u21RNA\t\S+\t\S+\t\S+', line)):
                mch = re.search(r'u21RNA\t(\S+)\t\S+\t\S+', line) # sense
                u21 = float(mch.group(1))
            if (re.search(r'^star-miRNA', line)):
                break
    nons = geex - struc
        
    print ('''total %.2f
sense struc %.2f
non-struc %.2f
sense coding %.2f
anti coding %.2f
total coding %.2f
sense miRNA %.2f
sense 21U-RNA %.2f\n''' % (geex, struc, nons, cose, coan, coto, mir, u21))
        
    if (stan == 'mir'):
        ratio = 1000000/mir
    elif (stan == 'cose'):
        ratio = 5000000/cose
    elif (stan == 'coan'):
        ratio = 2000000/coan
    elif (stan == 'coto'):
        ratio = 5000000/coto
    elif (stan == 'nons'):
        ratio = 5000000/nons
    elif (stan == 'tot'):
        ratio = 5000000/geex
    elif (stan == '21U'):
        ratio = 500000/u21
    
    print ('Normalization ratio', ratio, '\n\n')
    
    return ratio



def cdna_sum(sam):
    csrge = {}; csrtr = {}
    ra = norm(sam, stan)
    #### sometimes may not need csrge
    with open('csr1_targets_at_least_three_overlap.txt', 'rt') as f:
        for line in f:
            n1, n2, n3 = line.rstrip().split('\t')
            csrge[n1+'\t'+n2] = 1
    
    if (utr == 'CDS'):
        fh = 'ceWS215_CDS_pos.txt'
    else:
        fh = 'ceWS215_CDS_' + utr + '_filtered.txt'
        
    with open(fh, 'rt') as f:
        for line in f:
            line = line.rstrip()
            if line == '':
                break
            ids, pos = line.split('|')
            idl = ids.split('\t') # id list
            idx = idl[1]+'\t'+idl[3]
            po = sorted(pos.split())
            #if (idx not in csrge.keys()): # only consider csr-1 targets
                #continue
            csrtr[idx] = po[0]+'\t'+po[-1]

    csrge.clear()
    
    with open(sam+'/'+sam+'_17_cdna_coding_normed.txt', 'rt') as f:
        s = {}; a = {}; se = {}; an = {}
        for line in f:
            line = line.rstrip()
            a1 = line.split('\t')
            if line.startswith('>'):
                on = 0
                sr = a1[6]
                idx = a1[1]+'\t'+a1[3]
                if (idx in csrtr.keys()):
                    l, r = csrtr[idx].split('\t')
                    l = float(l)
                    r = float(r)
                    se[idx] = {}
                    an[idx] = {}
                    on = 1
                continue
            if (on != 1):
                continue
            if (int(a1[2]) < l or int(a1[2]) > r):
                continue
            if (int(a1[2])+int(a1[3]) < l or int(a1[2])+int(a1[3]) > r):
                continue
            if (a1[1] == sr):
                s[line] = int(a1[4])/int(a1[5])
                se[idx][line] = int(a1[4])/int(a1[5])
            else:
                a[line] = int(a1[4])/int(a1[5])
                an[idx][line] = int(a1[4])/int(a1[5])

    fh1 = open(sam+'/sum_'+sam+'_all_'+utr+'_17_cdna_coding_'+stan+'.txt', 'wt')
        
    #for idx in sorted(se.keys(),key=lambda s: s.lower()):
    for idx in sorted(se.keys()):
        al_n = 0; se_n = 0; an_n = 0
        for fe in se[idx].keys():
            sh = 1
            if fe in an[idx].keys():
                sh = 0.5
            se_n+= se[idx][fe]*ra*sh
            al_n+= se[idx][fe]*ra*sh
        for fe in an[idx].keys():
            sh = 1
            if fe in se[idx].keys():
                sh = 0.5
            an_n+= an[idx][fe]*ra*sh
            al_n+= an[idx][fe]*ra*sh
        se_n = str(round(se_n, 3))
        an_n = str(round(an_n, 3))
        al_n = str(round(al_n, 3))
            
        fh1.write(idx+'\t'+se_n+'\t'+an_n+'\t'+al_n+'\n')
        #print >> fh1, idx+'\t'+se_n+'\t'+an_n+'\t'+al
    fh1.close()
    
    fh2 = open('sum_'+utr+'_read_CSR1_targets_'+stan+'.txt', 'at')
    al_n = 0; se_n = 0; an_n = 0
    for fe in s.keys():
        sh = 1
        if fe in a.keys():
            sh = 0.5
        se_n+= s[fe]*ra*sh
        al_n+= s[fe]*ra*sh
    for fe in a.keys():
        sh = 1
        if fe in s.keys():
            sh = 0.5
        an_n+= a[fe]*ra*sh
        al_n+= a[fe]*ra*sh
    se_n = str(round(se_n, 3))
    an_n = str(round(an_n, 3))
    al_n = str(round(al_n, 3))
    fh2.write(sam+'\t'+se_n+'\t'+an_n+'\t'+al_n+'\n')
    fh2.close()
    
if __name__ == '__main__':
    main()

Sample names (with _all) seperated with a comma:
0320tocsr1om_all,0320toom_all
Normalization standard:
mir
5utr, 3utr, or CDS
3utr
total 10310921.00
sense struc 972969.38
non-struc 9337951.62
sense coding 215111.03
anti coding 4064389.66
total coding 4279500.69
sense miRNA 1264368.17
sense 21U-RNA 1171800.61

Normalization ratio 0.7909088695265083 


total 6762488.00
sense struc 336050.82
non-struc 6426437.18
sense coding 100989.96
anti coding 3382585.06
total coding 3483575.02
sense miRNA 1015206.00
sense 21U-RNA 205742.87

Normalization ratio 0.9850217591306591 




In [48]:
utr = 'CDS'; gene = 'gene'
if (utr == 'CDS'):
    print utr
print gene

CDS
gene


In [25]:
utr = 'h|x'
id, po = utr.split('|')
print id

h


In [29]:
utr = 'h\tx'
print utr

h	x


In [35]:
po = [1,3,5,'t', 'g']
po = sorted(po, reverse = True)
print po

['t', 'g', 5, 3, 1]


In [78]:
ids = 'csr1\tF04\tFFFF'
id = ids.split('\t')
print id
csrge = {}
csrge['csr1\tF04'] = 5
print csrge['csr1\tF04']
print csrge[id[0]+'\t'+id[1]]
x = id[0]+'\t'+id[1]
print x


if (x not in csrge.keys()):
    print 'no'
       

for x in sorted(csrge.keys()):
    print x

    
if (type(csrge) is dict):
    print 'cool'
    
       
csrge.clear()        
print csrge




['csr1', 'F04', 'FFFF']
5
5
csr1	F04
csr1	F04
cool
{}


TypeError: 'dict' object is not callable

In [50]:
ids = 'csr1\tF04\tFFFF'
id = ids.split('\t')
print id[-1]

FFFF


In [86]:
definitions = {}
 
# and, as above, we have several dictionaries in this dictionary
#definitions['a'] = {}
#definitions['aTableAlphabeticall'] = {}
 
print definitions
 
# and each dictionary is full of definitions
if (type(definitions['a']) is dict):
    print 'yes'
else:
    print 'no'
#definitions['aTableAlphabeticall']['statue'] = 'interesting'
#definitions['aTableAlphabeticall']['vapor'] = 'moisture, ayre, hote breath, or reaking'


#print definitions

{}


KeyError: 'a'

In [66]:
se = 2
se+=1
print se

3


In [2]:
m = 3.1415926
y = round(m,3)
print y

3.142


In [4]:
sam = 'omcsr1to'
print ('sum_'+sam+'_all.txt')

sum_omcsr1to_all.txt


In [6]:
import os
os.getcwd()

'/Users/DONG/Documents/Python/Python_world/LABDATA/python_rewriting'

In [12]:
direc = 'folder'
sam = 'test'
fh1 = open(direc+'/'+sam+'.txt', 'r')
for line in fh1:
    print line.strip()

fh1.close()

puf-11	Y73B6BL.38	323.443	153.8368	0.6777	Not	csr	Not	Not
ced-4	C35D10.9a	320.7128	116.7556	0.7331	Not	csr	Not	alg
F22D6.2	F22D6.2.2	320.5878	163.9756	0.6616	Not	csr	Not	Not
F08F8.2	F08F8.2	320.4834	131.4709	0.7091	Not	csr	Not	Not
R02F2.1	R02F2.1a.1	319.6229	155.4857	0.6727	Not	csr	Not	Not


In [14]:
su = 0
su+=1
print su

1


In [3]:
import re
expression = '[AP]{1,2}D'
pattern = re.compile(expression)
print pattern

seq = "RPAD"
match = pattern.search(seq)
print match

seq[match.start():match.end()]

'PAD'

In [2]:
line = 'stop\n'
print line

import re
line = re.sub(r'\s+$', r'', line)
print line

stop

stop


In [25]:
import re
geex = 0
line = 'geex 5173121 5137800 10310921'
if (re.search(r'^geex\s\d+\s\d+\s(\d+)', line)):
    m = re.search(r'^geex\s\d+\s(\d+)\s(\d+)', line)
    x = float(m.group(1))
    print x

5137800.0


In [27]:
print '''total geex
sense struc struc
non-struc nons
sense coding cose
anti coding coan
total coding coto
sense miRNA mir
sense 21U-RNA u21\n'''

stan = 'mir'
mir = 20000
if (stan == 'mir'):
        ratio = 1000000/mir
        print ratio

total geex
sense struc struc
non-struc nons
sense coding cose
anti coding coan
total coding coto
sense miRNA mir
sense 21U-RNA u21

50


In [26]:
stan = 'mir'
mir = 20000
if (stan == 'mir'):
        ratio = 1000000/mir
        print ratio

50


In [54]:
def norm(stan):
    geex = 10310921
    struc = 972969.38
    nons = geex - struc
    cose = 215111.03
    coan = 4064389.66
    coto = 4279500.69
    mir = 1264368.17
    u21 = 1171800.61
    print '''total %f
sense struc %20.2f
non-struc %s
sense coding %.2f
anti coding %.2f
total coding %.2f
sense miRNA %.2f
sense 21U-RNA %.2f\n''' % (geex, struc, nons, cose, coan, coto, mir, u21)
    
    if (stan == 'mir'):
        ratio = 1000000/mir
    elif (stan == 'cose'):
        ratio = 5000000/cose
    elif (stan == 'coan'):
        ratio = 2000000/coan
    elif (stan == 'coto'):
        ratio = 5000000/coto
    elif (stan == 'nons'):
        ratio = 5000000/nons
    elif (stan == 'tot'):
        ratio = 5000000/geex
    elif (stan == '21U'):
        ratio = 500000/u21
    
    print 'Normalization ratio', round(ratio,4), '\n\n'
    return ratio


stan = 'coto'
print norm(stan)*3

total 10310921.000000
sense struc            972969.38
non-struc 9337951.62
sense coding 215111.03
anti coding 4064389.66
total coding 4279500.69
sense miRNA 1264368.17
sense 21U-RNA 1171800.61

Normalization ratio 1.1684 


3.5050818043


In [1]:
print 'Sample names (no _all) seperated with a comma:'

Sample names (no _all) seperated with a comma:


In [3]:
sam = int(raw_input(m))
x = sam*3
print x

NameError: name 'm' is not defined

In [ ]:
sam = raw_input()

In [9]:
sam = raw_input('Sample names (no _all) seperated with a comma:\n')
len(sam)

Sample names (no _all) seperated with a comma:
34


2

In [11]:
len('34\t')

3

In [12]:
ids = 'x y z'
ids = ids.split(' ')
print ids
ids = ids[1]+'\t'+ids[2]
print ids

['x', 'y', 'z']
y	z


In [35]:
sam = 'x'
sam = re.split(r'\s*\,+\s*', sam)
print sam

['x']


In [47]:
a = 'm'
print a+'\t'+11.5

TypeError: cannot concatenate 'str' and 'float' objects

In [2]:
m = ['1','2','7','4','5']
m = sorted(m)
print (m)

['1', '2', '4', '5', '7']


In [3]:
m = '5\t10'
l,r = m.split('\t')
print l, r
if (l < 10):
    print 'yes'
else:
    print 'no'

5 10
no


In [3]:
int(3)/int(4)

0.75

In [8]:
hs = {'a':{'x':1, 'y':7}, 'b': 11}

for x in hs.keys():
    if isinstance(hs[x],dict):
        for z in hs[x].keys():
            print (z)
    print (x)
    

b
x
y
a


In [19]:
hs = {'a':1, 'B': 11, 'c':3}

for x in sorted(hs.keys(), key=lambda s: s.lower()):
    print (x)
    
for x in sorted(hs.keys()):
    print (x)

a
B
c
B
a
c


In [29]:
x = 'a b c'
m = x.split()
for z in m:
    print (z)

a
b
c
